<a href="https://colab.research.google.com/github/ssgalitsky/PedalNerd/blob/main/PedalNerd_S1E1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

  

 | W$_Я$a${_P}$ **C** o$^{1_A}b$  |  <img width=4000px/>   |     $^{(C)\ 2020,\  Sergey\ Galitsky.}_{Brain,\ Music,\ Hatered}$ |
| :-------------------------------   |  :-----------------------------------:  | --------------------:  |


In [ ]:
#@markdown         ## **PedalNerd #s1e1**  
#@markdown ##   training WaveNet on Colab

#@markdown Parameters:

#@markdown - Use Google TPU option 
UseTPU = True  #@param {type: "boolean"}

#!rm -rf '/content/PedalNetRT'
#import os
#os.chdir('/content')
#!git clone https://github.com/ssgalitsky/PedalNetRT.git
!pip3 install torchvision
!pip3 install pytorch_lightning==0.9.0

if UseTPU:
  !curl https://raw.githubusercontent.com/pytorch/xla/master/contrib/scripts/env-setup.py -o pytorch-xla-env-setup.py
  !python3 pytorch-xla-env-setup.py --version nightly --apt-packages libomp5 libopenblas-dev
  !pip3 install cloud-tpu-client==0.10 https://storage.googleapis.com/tpu-pytorch/wheels/torch_xla-1.6-cp36-cp36m-linux_x86_64.whl

import os
import torch


use_cuda=True
if use_cuda and torch.cuda.is_available():
  net.cuda()

if UseTPU:
  assert os.environ['COLAB_TPU_ADDR']
  

In [9]:
assert os.environ['COLAB_TPU_ADDR']

In [10]:
#@markdown ##### Upload input .wav file
from google.colab import files

uploaded = files.upload()
for inFilename in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=inFilename, length=len(uploaded[inFilename])))
  


Saving s chain1.wav to s chain1 (1).wav
User uploaded file "s chain1.wav" with length 8368792 bytes


In [11]:
!ls

 attack.json		   's chain1.wav_16m'
'b chain1.wav'		   's chain1.wav_16m.wav'
 pytorch-xla-env-setup.py   torch-nightly-cp36-cp36m-linux_x86_64.whl
 sample_data		    torchvision-nightly-cp36-cp36m-linux_x86_64.whl
's chain1 (1).wav'	    torch_xla-nightly-cp36-cp36m-linux_x86_64.whl
's chain1.wav'


In [12]:
#@markdown ##### Upload output .wav file
from google.colab import files

uploaded = files.upload()
for outFilename in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=outFilename, length=len(uploaded[outFilename])))

Saving s chain4.wav to s chain4.wav
User uploaded file "s chain4.wav" with length 8368792 bytes


In [13]:
 !ls 
 print(inFilename)

 attack.json		   's chain1.wav_16m'
'b chain1.wav'		   's chain1.wav_16m.wav'
 pytorch-xla-env-setup.py  's chain4.wav'
 sample_data		    torch-nightly-cp36-cp36m-linux_x86_64.whl
's chain1 (1).wav'	    torchvision-nightly-cp36-cp36m-linux_x86_64.whl
's chain1.wav'		    torch_xla-nightly-cp36-cp36m-linux_x86_64.whl
s chain1.wav


In [15]:
#@markdown ##### Prepare data (convert to PCM 16bit LE and then merge to pickle)
sampleTime = 100e-3  #@param {type: "number"}


!ffmpeg -i "$inFilename" -acodec pcm_s16le -ac 1 -ar 41000 i_"$inFilename"
!ffmpeg -i "$outFilename" -acodec pcm_s16le -ac 1 -ar 41000 o_"$outFilename"

import pickle
from scipy.io import wavfile
import numpy as np


#os.chdir('/content/PedalNetRT')
#!python3 "prepare_data.py" "/content/PedalNetRT/data/s chain1.wav" "/content/PedalNetRT/data/s chain3.wav"

def prepareWavs():
    in_rate, in_data = wavfile.read("i_"+inFilename)
    out_rate, out_data = wavfile.read("o_"+outFilename)
#    assert in_rate == out_rate, "input  out_file must have same sample rate"

    sample_size = int(in_rate * sampleTime)
    length = len(in_data) - len(in_data) % sample_size

    x = in_data[:length].reshape((-1, 1, sample_size)).astype(np.float32)
    y = out_data[:length].reshape((-1, 1, sample_size)).astype(np.float32)

    split = lambda d: np.split(d, [int(len(d) * 0.6), int(len(d) * 0.8)])

    d = {}
    d["x_train"], d["x_valid"], d["x_test"] = split(x)
    d["y_train"], d["y_valid"], d["y_test"] = split(y)
    d["mean"], d["std"] = d["x_train"].mean(), d["x_train"].std()

    # standardize
    for key in "x_train", "x_valid", "x_test":
        d[key] = (d[key] - d["mean"]) / d["std"]

    pickle.dump(d, open("data.pickle", "wb"))


prepareWavs()


ffmpeg version 3.4.8-0ubuntu0.2 Copyright (c) 2000-2020 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.5.0-3ubuntu1~18.04)
  configuration: --prefix=/usr --extra-version=0ubuntu0.2 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --enable-gpl --disable-stripping --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librubberband --enable-librsvg --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvorbis --enable-libvpx --enable-libwavpack --enable-libwebp --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lib

In [16]:
!ls 

 attack.json		   's chain1.wav'
'b chain1.wav'		   's chain1.wav_16m'
 data.pickle		   's chain1.wav_16m.wav'
'i_s chain1.wav'	   's chain4.wav'
'o_s chain4.wav'	    torch-nightly-cp36-cp36m-linux_x86_64.whl
 pytorch-xla-env-setup.py   torchvision-nightly-cp36-cp36m-linux_x86_64.whl
 sample_data		    torch_xla-nightly-cp36-cp36m-linux_x86_64.whl
's chain1 (1).wav'


In [33]:
#@markdown ##### Train model
# Beware, checkpoint files not visible via Colab webui (-rw-r--r--) 

#import pytorch_lightning
#!python3 "train.py" --batch_size=32 --max_epochs=100 --learning_rate=3e-3 --num_channels=5 --kernel_size=4 --dilation_depth=8
#!python3 "train.py" --batch_size=32 --max_epochs=100 --learning_rate=3e-3 --num_channels=5 

numChannels=5          #@param {type: "number"}
dilationDepth=10       #@param {type: "number"}
numRepeat=1            #@param {type: "number"}
    
    # filter_width=kernel_size
kernelSize=3           #@param {type: "number"}

batchSize=64           #@param {type: "number"}
learningRate=3e-3      #@param {type: "number"}

maxEpochs=100         #@param {type: "number"}
gpuCores=0                  #@param {type: "number"}
tpuCores=8             #@param {type: "number"}
data="data.pickle"

import pytorch_lightning as pl

#from model import PedalNet

from pytorch_lightning.callbacks import ModelCheckpoint

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader

#import pytorch_lightning as pl
import pickle
#import argparse


class CausalConv1d(torch.nn.Conv1d):
    def __init__(self,
                 in_channels,
                 out_channels,
                 kernel_size,
                 stride=1,
                 dilation=1,
                 groups=1,
                 bias=True):
        self.__padding = (kernel_size - 1) * dilation

        super(CausalConv1d, self).__init__(
            in_channels,
            out_channels,
            kernel_size=kernel_size,
            stride=stride,
            padding=self.__padding,
            dilation=dilation,
            groups=groups,
            bias=bias)

    def forward(self, input):
        result = super(CausalConv1d, self).forward(input)
        if self.__padding != 0:
            return result[:, :, :-self.__padding]
        return result
        

def _conv_stack(dilations, in_channels, out_channels, kernel_size):
    """
    Create stack of dilated convolutional layers, outlined in WaveNet paper:
    https://arxiv.org/pdf/1609.03499.pdf
    """
    return nn.ModuleList(
        [
            CausalConv1d(
                in_channels=in_channels,
                out_channels=out_channels,
                dilation=d,
                kernel_size=kernel_size,
            )
            for i, d in enumerate(dilations)
        ]
    )


class WaveNet(nn.Module):
    def __init__(self, num_channels, dilation_depth, num_repeat, kernel_size=2):
        super(WaveNet, self).__init__()
        dilations = [2 ** d for d in range(dilation_depth)] * num_repeat
        internal_channels = int(num_channels*2)
        self.hidden = _conv_stack(dilations, num_channels, internal_channels, kernel_size)
        self.residuals = _conv_stack(dilations, num_channels, num_channels, 1)
        self.input_layer = CausalConv1d(
            in_channels=1,
            out_channels=num_channels,
            kernel_size=1,
        )

        self.linear_mix = nn.Conv1d(
            in_channels=num_channels * dilation_depth * num_repeat,
            out_channels=1,
            kernel_size=1,
        )
        self.num_channels = num_channels

    def forward(self, x):
        out = x
        skips = []
        out = self.input_layer(out)

        for hidden, residual in zip(
            self.hidden, self.residuals
        ):
            x = out
            out_hidden = hidden(x)

            # gated activation
            #   split (32,16,3) into two (16,16,3) for tanh and sigm calculations
            out_hidden_split = torch.split(out_hidden, self.num_channels, dim=1)  
            out = torch.tanh(out_hidden_split[0]) * torch.sigmoid(out_hidden_split[1])

            skips.append(out)

            out = residual(out)
            out = out + x[:, :, -out.size(2) :]  

        # modified "postprocess" step:
        out = torch.cat([s[:, :, -out.size(2) :] for s in skips], dim=1)
        out = self.linear_mix(out)
        return out


def error_to_signal(y, y_pred):
    """
    Error to signal ratio with pre-emphasis filter:
    https://www.mdpi.com/2076-3417/10/3/766/htm
    """
    y, y_pred = pre_emphasis_filter(y), pre_emphasis_filter(y_pred)
    return (y - y_pred).pow(2).sum(dim=2) / (y.pow(2).sum(dim=2) + 1e-10)


def pre_emphasis_filter(x, coeff=0.95):
    return torch.cat((x[:, :, 0:1], x[:, :, 1:] - coeff * x[:, :, :-1]), dim=2)


class PedalNet(pl.LightningModule):
#    def __init__(self, hparams):
    def __init__(self):
        super(PedalNet, self).__init__()
        self.wavenet = WaveNet(
                num_channels=numChannels,
                dilation_depth=dilationDepth,
                num_repeat=numRepeat,
                kernel_size=kernelSize,
              )

  #      self.hparams = hparams

    def prepare_data(self):
        ds = lambda x, y: TensorDataset(torch.from_numpy(x), torch.from_numpy(y))
        data = pickle.load(open("data.pickle", "rb"))
        self.train_ds = ds(data["x_train"], data["y_train"])
        self.valid_ds = ds(data["x_valid"], data["y_valid"])

    def configure_optimizers(self):
        return torch.optim.Adam(
            self.wavenet.parameters(), lr=learningRate
        )

    def train_dataloader(self):
        return DataLoader(
            self.train_ds,
            shuffle=True,
            batch_size=batchSize,
            num_workers=4,
        )

    def val_dataloader(self):
        return DataLoader(
            self.valid_ds, batch_size=batchSize, num_workers=4
        )

    def forward(self, x):
        return self.wavenet(x)

    def training_step(self, batch, batch_idx):
        x, y = batch
        y_pred = self.forward(x)
        loss = error_to_signal(y[:, :, -y_pred.size(2) :], y_pred).mean()
        logs = {"loss": loss}
        return {"loss": loss, "log": logs}

    def validation_step(self, batch, batch_idx):
        x, y = batch
        y_pred = self.forward(x)
        loss = error_to_signal(y[:, :, -y_pred.size(2) :], y_pred).mean()
        return {"val_loss": loss}

    def validation_epoch_end(self, outs):
        avg_loss = torch.stack([x["val_loss"] for x in outs]).mean()
        logs = {"val_loss": avg_loss}
        return {"avg_val_loss": avg_loss, "log": logs}

checkpoint_callback = ModelCheckpoint(
    save_top_k=5,
    monitor='val_loss',
    mode='min',
    period=100
)

model = PedalNet()
if UseTPU:
     trainer = pl.Trainer(
        # max_epochs=args.max_epochs, gpus=args.gpus, row_log_interval=100
        # The following line is for use with the Colab notebook when training on TPUs.
        # Comment out the above line and uncomment the below line to use.
          max_epochs=maxEpochs, tpu_cores=tpuCores, row_log_interval=100, checkpoint_callback=ModelCheckpoint()
        )
     trainer.fit(model)
else:
     trainer = pl.Trainer(
        max_epochs=maxEpochs, gpus=gpuCores, row_log_interval=100, checkpoint_callback=ModelCheckpoint()
        )     
     trainer.fit(model) 





GPU available: False, used: False
TPU available: True, using: 8 TPU cores
training on 8 TPU cores
Exception in device=TPU:0: Cannot replicate if number of devices (1) is different from 8
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/torch_xla/distributed/xla_multiprocessing.py", line 330, in _mp_start_fn
    _start_fn(index, pf_cfg, fn, args)
Exception in device=TPU:2: Cannot replicate if number of devices (1) is different from 8
Exception in device=TPU:1: Cannot replicate if number of devices (1) is different from 8
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/torch_xla/distributed/xla_multiprocessing.py", line 323, in _start_fn
    _setup_replication()
  File "/usr/local/lib/python3.6/dist-packages/torch_xla/distributed/xla_multiprocessing.py", line 316, in _setup_replication
    xm.set_replication(device, [device])
  File "/usr/local/lib/python3.6/dist-packages/torch_xla/core/xla_model.py", line 315, in set_rep

ProcessExitedException: ignored

In [28]:
!ls -d /content/lightning_logs/version_0/*

#!chmod -R 755 lightning_logs 


/content/lightning_logs/version_0/checkpoints
/content/lightning_logs/version_0/events.out.tfevents.1602804568.8c7ab60b5554.2382.0
/content/lightning_logs/version_0/hparams.yaml


In [ ]:
#@title 5. convert torch model (ckpt) to plugin model (json) 
# The .ckpt model must be converted to a .json model to run in the plugin. Usage:

#!python3 convert_pedalnet_to_wavnetva.py --model=your_trained_model.ckpt
#!python3 convert_pedalnet_to_wavenetva.py --model=/content/PedalNetRT/lightning_logs/version_1/checkpoints/epoch=99.ckpt

import json
#import torch

#from model import PedalNet

def convert(args):
    # Permute tensors to match Tensorflow format with .permute(a,b,c):
    a, b, c = 2, 1, 0  # Pytorch uses (out_channels, in_channels, kernel_size), TensorFlow uses (kernel_size, in_channels, out_channels)
    model = PedalNet.load_from_checkpoint(checkpoint_path=args.model)

    sd = model.state_dict()

    # Get hparams from model
    hparams = model.hparams
    residual_channels = hparams["num_channels"]
    filter_width = hparams["kernel_size"]
    dilations = [2 ** d for d in range(hparams["dilation_depth"])] * hparams["num_repeat"]

    data_out = {"activation": "gated", 
                "output_channels": 1, 
                "input_channels": 1, 
                "residual_channels": residual_channels, 
                "filter_width": filter_width, 
                "dilations": dilations, 
                "variables": []}

    # Use pytorch model data to populate the json data for each layer
    for i in range(-1, len(dilations) + 1):
        # Input Layer
        if i == -1: 
            data_out["variables"].append({"layer_idx":i,
                                        "data":[str(w) for w in (sd['wavenet.input_layer.weight']).permute(a,b,c).flatten().numpy().tolist()],
                                        "name":"W"})
            data_out["variables"].append({"layer_idx":i,
                                        "data":[str(b) for b in (sd['wavenet.input_layer.bias']).flatten().numpy().tolist()],
                                        "name":"b"})
        # Linear Mix Layer
        elif  i == len(dilations):  
            data_out["variables"].append({"layer_idx":i,
                                        "data":[str(w) for w in (sd['wavenet.linear_mix.weight']).permute(a,b,c).flatten().numpy().tolist()], 
                                        "name":"W"})

            data_out["variables"].append({"layer_idx":i,
                                        "data":[str(b) for b in (sd['wavenet.linear_mix.bias']).numpy().tolist()],
                                        "name":"b"})
        # Hidden Layers
        else:
            data_out["variables"].append({"layer_idx":i,
                                    "data":[str(w) for w in sd['wavenet.hidden.' + str(i) + '.weight'].permute(a,b,c).flatten().numpy().tolist()],
                                    "name":"W_conv"})  
            data_out["variables"].append({"layer_idx":i,
                                        "data":[str(b) for b in sd['wavenet.hidden.' + str(i) + '.bias'].flatten().numpy().tolist()],
                                        "name":"b_conv"})            
            data_out["variables"].append({"layer_idx":i,
                                        "data":[str(w2) for w2 in sd['wavenet.residuals.' + str(i) + '.weight'].permute(a,b,c).flatten().numpy().tolist()],
                                        "name":"W_out"})
            data_out["variables"].append({"layer_idx":i,
                                        "data":[str(b2) for b2 in sd['wavenet.residuals.' + str(i) + '.bias'].flatten().numpy().tolist()],
                                        "name":"b_out"})

    # output final dictionary to json file
    with open('converted_model.json', 'w') as outfile:
        json.dump(data_out, outfile)



2020-10-15 00:21:38.871550: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1


In [ ]:
!python test.py --model=/content/PedalNetRT/lightning_logs/version_1/checkpoints/epoch=99.ckpt

2020-10-15 00:21:56.102070: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1


In [ ]:
!python plot_wav.py

Error to signal (with pre-emphasis filter):  0.86500657
Error to signal (no pre-emphasis filter):  0.8650066
Creating spectrogram data..
plot_wav.py:116: RuntimeWarning: divide by zero encountered in log10
  plt.pcolormesh(times, frequencies, 10*np.log10(spectrogram))
<Figure size 1300x800 with 3 Axes>
<Figure size 1200x800 with 2 Axes>


In [ ]:
#@title 6. convert model from TPU to CPU format
import torch_xla
import torch_xla.core.xla_model as xm

from model import PedalNet
# Change path below to match model file
model = PedalNet.load_from_checkpoint('lightning_logs/version_0/checkpoints/epoch=1.ckpt')

# xm.save(model.state_dict(), 'tpu_to_cpu.ckpt') 
xm.save(model, 'tpu_to_cpu_model.ckpt') 
# issues loading model, see pytorch_lightning issues #2303 and #3044 (Might be completed by now)

# TODO: Add code to be able to load saved model sucessfully
